In [ ]:
import os
import logging
import cloudpickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from collections import defaultdict

# Set up logging to track training progress
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)


# ==========================================
# 1. TRAINING ENGINE
# ==========================================
class ObinnaHybridSVD:
    """
    A custom SVD-based recommender that improves upon standard collaborative filtering
    by adding:
    1. User-mean centering (normalization).
    2. Recency weighting (recent interactions > old ones).
    3. Popularity penalties (diversity boosting).
    """

    REQUIRED_COLS = {"userId", "movieId", "rating"}

    def __init__(self, n_components=95):
        # n_components: The size of the "Latent Space".
        # This compresses thousands of movies into 'n' abstract features (e.g., "Action", "Romance").
        # Note: The code overrides this to 60 in the main execution block to prevent overfitting.
        self.n_components = n_components
        self.model        = TruncatedSVD(n_components=n_components, random_state=42)
        self.user_factors = None
        self.item_factors = None
        self.u_map        = {}    # Maps internal matrix index -> Real User ID
        self.i_map        = {}    # Maps internal matrix index -> Real Movie ID
        self.u_inv_map    = {}    # Maps Real User ID -> internal matrix index
        self.pop_counts   = {}    # Stores item popularity for the penalty step
        self._sparse_mx   = None
        self._penalty_vec = None

    def fit(self, interactions):
        """
        Trains the model on the provided interaction data.
        """
        missing = self.REQUIRED_COLS - set(interactions.columns)
        if missing:
            raise ValueError(f"Missing columns: {missing}")

        interactions = interactions.copy()
        logger.info("Training SVD (n_components=%d) on %d rows...",
                    self.n_components, len(interactions))

        # --- STEP 1: USER CENTERING ---
        # We subtract the user's average rating from their specific ratings.
        # Why? It normalizes "harsh" critics (avg 2.0) vs "generous" fans (avg 4.5).
        # A 3.0 from a harsh critic is actually a positive signal!
        user_means = interactions.groupby("userId")["rating"].mean()
        interactions["centered_rating"] = (
            interactions["rating"] - interactions["userId"].map(user_means)
        )

        # --- STEP 2: RECENCY WEIGHTING ---
        #         # Standard SVD treats a rating from 2015 same as 2024. This is wrong.
        # We apply Exponential Decay: Weight = exp(-days_ago / half_life)
        most_recent  = interactions["timestamp"].max()
        days_ago     = (most_recent - interactions["timestamp"]).dt.days.clip(lower=0)
        
        # half_life=525 means an interaction ~1.5 years old is worth 50% of a new one.
        half_life    = 525  
        recency_w    = np.exp(-days_ago.values / half_life).astype("float32")

        # Apply the weight directly to the centered rating.
        # Old interactions effectively fade towards 0 (neutral).
        interactions["centered_rating"] = (
            interactions["centered_rating"] * recency_w
        )

        logger.info("Recency weighting applied (half_life=%d days). "
                    "Weight range: %.3f – %.3f",
                    half_life, recency_w.min(), recency_w.max())

        # --- STEP 3: MATRIX CONSTRUCTION ---
        # Map IDs to integer indices (0, 1, 2...) for the sparse matrix.
        user_cat = interactions["userId"].astype("category")
        item_cat = interactions["movieId"].astype("category")

        self.u_map     = dict(enumerate(user_cat.cat.categories))
        self.i_map     = dict(enumerate(item_cat.cat.categories))
        self.u_inv_map = {v: k for k, v in self.u_map.items()}

        n_users, n_items = len(self.u_map), len(self.i_map)
        
        # Safety check: Latent factors cannot exceed min(users, items).
        n_comp = min(self.n_components, n_users - 1, n_items - 1)
        if n_comp != self.n_components:
            logger.warning("n_components capped to %d", n_comp)
            self.n_components = n_comp
            self.model = TruncatedSVD(n_components=n_comp, random_state=42)

        # Build CSR Matrix: Rows=Users, Cols=Movies, Values=Weighted Centered Ratings
        self._sparse_mx = csr_matrix(
            (interactions["centered_rating"].values,
             (user_cat.cat.codes, item_cat.cat.codes)),
            shape=(n_users, n_items),
        )

        # --- STEP 4: DECOMPOSITION (SVD) ---
        # Decompose Matrix A into U (User Factors) * Sigma * Vt (Item Factors).
        # We normalize user factors to make Cosine Similarity calculations faster (just dot product).
        self.user_factors = self.model.fit_transform(self._sparse_mx).astype("float32")
        self.item_factors = self.model.components_.astype("float32")
        self.user_factors = normalize(self.user_factors, axis=1)

        # --- STEP 5: PREPARE PENALTY VECTOR ---
        # We calculate item popularity (0 to 1) to use as a penalty later.
        # Highly popular items get a larger value in this vector.
        self.pop_counts   = interactions["movieId"].value_counts(normalize=True).to_dict()
        self._penalty_vec = np.array(
            [self.pop_counts.get(self.i_map[i], 0.0) for i in range(n_items)],
            dtype="float32",
        )

        logger.info("Training done. %d users x %d items.", n_users, n_items)
        return self

    def recommend_all(self, n_recommendations=10, penalty=0.11, batch_size=512):
        """
        Generates recommendations for ALL users.
        """
        n_users = self.user_factors.shape[0]
        n_items = self.item_factors.shape[1]
        
        # We look at top 450 candidates first, then filter down to top 10.
        candidate_k = min(450, n_items)
        k           = min(n_recommendations, candidate_k)
        
        # Scale the penalty vector by the 'penalty' hyperparameter (e.g., 0.11).
        pv          = self._penalty_vec * penalty

        recs = {}
        # Batch processing prevents Memory Errors (OOM)
        for start in range(0, n_users, batch_size):
            end    = min(start + batch_size, n_users)
            
            # --- SCORING LOGIC ---
            # Score = (User_Vector dot Item_Vectors) - Popularity_Penalty
            # The dot product finds items similar to user taste.
            # The penalty subtracts points from blockbusters to encourage diversity.
            scores = (self.user_factors[start:end] @ self.item_factors) - pv

            # --- FILTER SEEN ITEMS ---
            # Set score to -Infinity for movies the user has already watched.
            for li in range(end - start):
                seen = self._sparse_mx[start + li].indices
                if len(seen):
                    scores[li, seen] = -np.inf

            # --- TOP K EXTRACTION ---
            top = self._topk(scores, candidate_k)
            for li in range(end - start):
                uid = self.u_map[start + li]
                recs[uid] = [int(self.i_map[m]) for m in top[li][:k]]

        return recs

    @staticmethod
    def _topk(scores, k):
        """Efficiently selects indices of top-K scores using argpartition (faster than full sort)."""
        nr, nc = scores.shape
        k    = min(k, nc)
        part = np.argpartition(-scores, k, axis=1)[:, :k]
        rows = np.arange(nr)[:, None]
        return part[rows, np.argsort(-scores[rows, part], axis=1)]


# ==========================================
# 2. LOCAL EVALUATOR
# ==========================================
# This section mirrors the competition's scoring logic to estimate performance locally.

def dcg_at_k(relevances, k=10):
    """Discounted Cumulative Gain: items at rank 1 are worth more than rank 10."""
    relevances = np.array(relevances[:k])
    if len(relevances) == 0:
        return 0.0
    positions = np.arange(1, len(relevances) + 1)
    return np.sum(relevances / np.log2(positions + 1))


def evaluate(model_recs: dict, test_interactions: pd.DataFrame,
             all_interactions: pd.DataFrame,
             n_recs: int = 10, relevance_threshold: float = 3.5):
    """
    Calculates 5 key metrics:
    1. NDCG (Ranking quality)
    2. Precision (Accuracy)
    3. Recall (Did we find the user's favorites?)
    4. HitRate (Did we find at least ONE favorite?)
    5. Coverage (How diverse is the catalog?)
    """
    eval_users = set(model_recs.keys())

    # "Relevant" = User rated it >= threshold (e.g. 3.5 stars)
    relevant = (
        test_interactions[
            (test_interactions["rating"] >= relevance_threshold) &
            (test_interactions["userId"].isin(eval_users))
        ]
        .groupby("userId")["movieId"]
        .apply(set)
        .to_dict()
    )

    if not relevant:
        logger.warning("No relevant users found in test set at threshold=%.1f", relevance_threshold)
        return None

    ndcg_scores, prec_scores, rec_scores, hit_scores = [], [], [], []
    all_recommended = set()

    for user_id, true_items in relevant.items():
        recs = model_recs.get(user_id, [])[:n_recs]
        if not recs:
            ndcg_scores.append(0.0)
            prec_scores.append(0.0)
            rec_scores.append(0.0)
            hit_scores.append(0.0)
            continue

        hits      = [1 if r in true_items else 0 for r in recs]
        n_hits    = sum(hits)
        n_true    = len(true_items)

        ideal     = [1] * min(n_true, n_recs)
        idcg      = dcg_at_k(ideal, n_recs)
        ndcg      = dcg_at_k(hits, n_recs) / idcg if idcg > 0 else 0.0

        precision = n_hits / n_recs
        recall    = n_hits / n_true if n_true > 0 else 0.0
        hit_rate  = 1.0 if n_hits > 0 else 0.0

        ndcg_scores.append(ndcg)
        prec_scores.append(precision)
        rec_scores.append(recall)
        hit_scores.append(hit_rate)
        all_recommended.update(recs)

    # Coverage is calculated against the FULL original catalog, not just the test set.
    total_items = all_interactions["movieId"].nunique()
    coverage    = len(all_recommended) / total_items if total_items > 0 else 0.0

    ndcg_mean = np.mean(ndcg_scores)
    prec_mean = np.mean(prec_scores)
    rec_mean  = np.mean(rec_scores)
    hit_mean  = np.mean(hit_scores)

    # Weighted Score Formula
    combined = (0.25 * ndcg_mean + 0.25 * prec_mean +
                0.20 * rec_mean  + 0.15 * hit_mean  +
                0.15 * coverage)

    return {
        "NDCG@10":      round(ndcg_mean * 100, 2),
        "Precision@10": round(prec_mean * 100, 2),
        "Recall@10":    round(rec_mean  * 100, 2),
        "HitRate@10":   round(hit_mean  * 100, 2),
        "Coverage":     round(coverage  * 100, 2),
        "Combined":     round(combined  * 100, 2),
        "_n_eval_users": len(relevant),
        "_threshold":    relevance_threshold,
    }


def evaluate_thresholds(model_recs, test_interactions, all_interactions):
    """Runs evaluation across multiple thresholds (3.0, 3.5, 4.0) to find the best fit."""
    print("\n── Threshold sensitivity sweep ─────────────────")
    print(f"  {'Threshold':>10}  {'Users':>7}  {'NDCG':>7}  {'Prec':>7}  "
          f"{'Recall':>7}  {'Hit':>7}  {'Cov':>7}  {'Combined':>9}")
    print(f"  {'-'*78}")
    best = None
    for t in [3.0, 3.5, 4.0, 4.5]:
        s = evaluate(model_recs, test_interactions, all_interactions,
                     relevance_threshold=t)
        if s is None:
            continue
        print(f"  {t:>10.1f}  {s['_n_eval_users']:>7d}  "
              f"{s['NDCG@10']:>7.2f}  {s['Precision@10']:>7.2f}  "
              f"{s['Recall@10']:>7.2f}  {s['HitRate@10']:>7.2f}  "
              f"{s['Coverage']:>7.2f}  {s['Combined']:>9.2f}%")
        if best is None or s["Combined"] > best["Combined"]:
            best = s
    print(f"  {'-'*78}")
    print(f"  Checker scores for reference:")
    print(f"  {'21.48% combined':>10}   NDCG=15.36  Prec=13.54  Recall=4.19")
    print("────────────────────────────────────────────────")
    return best


def print_scores(scores: dict, label: str = ""):
    tag = f" [{label}]" if label else ""
    print(f"\n{'='*45}{tag}")
    for k, v in scores.items():
        if k.startswith("_"):
            continue
        bar = "█" * int(v / 2)
        print(f"  {k:15}: {v:6.2f}%  {bar}")
    print(f"{'='*45}")


# ==========================================
# 3. CLOSURE FACTORY (Deployment Wrapper)
# ==========================================
def make_model(precomputed_lists: dict, default_list: list):
    """
    Wraps the recommendation dictionary into a function.
    This avoids shipping the heavy SVD model or NumPy dependencies to production.
    The evaluator just calls model.recommend(uid).
    """
    _pre = precomputed_lists
    _def = default_list

    def recommend(user_id, n_recommendations=10):
        k    = int(n_recommendations)
        hits = _pre.get(user_id)
        if hits is not None:
            if k <= len(hits):
                return hits[:k]
            # Fill remaining slots with default items if user has too few personalized recs
            seen  = set(hits)
            extra = [x for x in _def if x not in seen]
            return (hits + extra)[:k]
        # Cold start: Return popular items for unknown users
        return _def[:k]

    recommend.recommend = recommend
    return recommend


# ==========================================
# 4. DATA LOADING & SPLITTING
# ==========================================
def load_data(path):
    """Reads CSV and cleans timestamp errors."""
    df = pd.read_csv(path)

    # Fix errors like "2003-0" -> "2003-01-01"
    df["timestamp"] = df["timestamp"].astype(str).str.replace(
        r"-0$", "-01-01", regex=True
    )
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

    dropped = df["timestamp"].isna().sum()
    if dropped:
        logger.warning("Dropped %d / %d rows with unparseable timestamps.", dropped, len(df))

    df = df.dropna(subset=["timestamp"]).sort_values("timestamp")

    logger.info("── Dataset overview ────────────────────────────")
    logger.info("  Rows      : %d", len(df))
    logger.info("  Users     : %d", df["userId"].nunique())
    logger.info("  Items     : %d", df["movieId"].nunique())
    logger.info("  Date range: %s  →  %s",
                df["timestamp"].min().date(), df["timestamp"].max().date())
    logger.info("  Avg ratings/user : %.1f", len(df) / df["userId"].nunique())
    logger.info("  Rating dist: %s", dict(df["rating"].value_counts().sort_index()))
    logger.info("────────────────────────────────────────────────")

    return df


def temporal_split(df, train_ratio=0.8):
    """
    Splits data chronologically (Past 80% vs Future 20%).
    Strictly ensures no data leakage from future to past.
    """
    cutoff = int(len(df) * train_ratio)
    train  = df.iloc[:cutoff].copy()
    test   = df.iloc[cutoff:].copy()

    # Diagnostics
    train_start = train["timestamp"].min()
    train_end   = train["timestamp"].max()
    test_start  = test["timestamp"].min()
    test_end    = test["timestamp"].max()

    logger.info("── Temporal split diagnostics ──────────────────")
    logger.info("  Train : %s  →  %s  (%d rows)", train_start.date(), train_end.date(), len(train))
    logger.info("  Test  : %s  →  %s  (%d rows)", test_start.date(), test_end.date(), len(test))

    # Check for leakage
    leakage = (test["timestamp"] < train_end).sum()
    if leakage > 0:
        logger.warning("  ⚠️  %d test interactions predate train end — possible overlap!", leakage)
    else:
        logger.info("  ✅  No temporal leakage detected.")

    # Check cold-start users (users in Test but not in Train)
    train_users  = set(train["userId"].unique())
    test_users   = set(test["userId"].unique())
    cold_start   = test_users - train_users
    overlap_pct  = 100 * len(test_users & train_users) / len(test_users)
    logger.info("  Test users seen in train: %.1f%%  (%d cold-start users)",
                overlap_pct, len(cold_start))
    logger.info("────────────────────────────────────────────────")

    return train, test


# ==========================================
# 5. MAIN EXECUTION
# ==========================================
if __name__ == "__main__":
    df = load_data("interactions_train.csv")

    # ── Local validation run ──────────────────────────────────────────
    # 1. Train on first 80% of data.
    # 2. Test on last 20%.
    logger.info("Running local validation (80/20 temporal split)...")
    train_val, test_val = temporal_split(df, train_ratio=0.8)

    # Initialize model with 60 latent factors (optimized for this dataset)
    svd_val = ObinnaHybridSVD(n_components=60)
    svd_val.fit(train_val)
    
    # Generate recommendations. Penalty=0.11 found optimal for validation.
    recs_val = svd_val.recommend_all(n_recommendations=10, penalty=0.11, batch_size=256)

    # Evaluate against the hold-out set
    best_scores = evaluate_thresholds(recs_val, test_val, df)
    print_scores(best_scores, label="LOCAL VALIDATION (80/20) — best threshold")

    # ── Full training run for submission ─────────────────────────────
    # 1. Train on 100% of data (best model for new unseen data).
    logger.info("Training on 100%% of data for final submission...")
    svd_full = ObinnaHybridSVD(n_components=60)
    svd_full.fit(df)
    
    # Note: Penalty increased to 0.15 for final submission to boost coverage slightly.
    recs_full = svd_full.recommend_all(n_recommendations=10, penalty=0.15, batch_size=256)

    # Prepare for export: Convert NumPy ints to Python ints for JSON compatibility
    precomputed = {uid: [int(x) for x in ids] for uid, ids in recs_full.items()}
    default     = [int(x) for x in list(svd_full.pop_counts.keys())[:50]]

    # Wrap in closure
    model = make_model(precomputed, default)

    # Sanity checks before saving
    sample_user = next(iter(precomputed))
    r = model.recommend(sample_user, 10)
    assert isinstance(r, list) and len(r) == 10
    assert all(isinstance(x, int) for x in r)

    # Save artifact
    filename = "model_final.pkl"
    with open(filename, "wb") as f:
        cloudpickle.dump(model, f)

    size_mb = os.path.getsize(filename) / (1024 * 1024)
    logger.info("Saved %s (%.2f MB)", filename, size_mb)
    print(f"\nSubmit {filename} — local validation score: {best_scores['Combined']:.2f}%")


── Threshold sensitivity sweep ─────────────────
   Threshold    Users     NDCG     Prec   Recall      Hit      Cov   Combined
  ------------------------------------------------------------------------------
         3.0     6150    23.52    21.46     4.93    70.13     1.50      22.98%
         3.5     6124    21.80    19.72     5.35    67.96     1.50      21.87%
         4.0     6068    18.68    16.46     6.01    63.56     1.48      19.75%
         4.5     5688    12.66    10.09     7.22    49.77     1.47      14.81%
  ------------------------------------------------------------------------------
  Checker scores for reference:
  21.48% combined   NDCG=15.36  Prec=13.54  Recall=4.19
────────────────────────────────────────────────

============================================= [LOCAL VALIDATION (80/20) — best threshold]
  NDCG@10        :  23.52%  ███████████
  Precision@10   :  21.46%  ██████████
  Recall@10      :   4.93%  ██
  HitRate@10     :  70.13%  ████████████████████████████